# Importacion y cruce de distintas bases de datos.
En este notebook se cruzará la base principal de peticiones de vulneración y restablecimiento de derechos con las bases de PARD, con un archivo de Reportes de Amenaza o Vulneración -RAVD- que contiene algunas variables importantes que no están en la base de peticiones. Además, se comprueba que los cruces sean consistentes y quede una base consolidada de peticiones con los campos necesarios para obtener la variable objetivo.

## Importación de librerias y configuración de lectura desde Azure SQL y Azure Storage Account

In [0]:
#Importación de librerías
import pandas as pd
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql.functions import expr, col, column, monotonically_increasing_id, desc, when
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from unicodedata import normalize

In [0]:
#Información sobre conexión a la base de datos.
jdbcHostname = dbutils.secrets.get('blobsecret','SQLHostname')
jdbcDatabase = dbutils.secrets.get('blobsecret','SQLDatabase')
jdbcPort = dbutils.secrets.get('blobsecret','SQLport')
jdbcuser = dbutils.secrets.get('blobsecret','SQLusername')
jdbcpass = dbutils.secrets.get('blobsecret','SQLpassword')
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)

In [0]:
# Información sobre conexión a contenedor del Storage Account
blobEndpoint=dbutils.secrets.get('blobsecret','blobendpoint')
blobKey=dbutils.secrets.get('blobsecret','blobkey')
try:
  dbutils.fs.mount(source=blobEndpoint,mount_point='/mnt/data1',extra_configs={'fs.azure.sas.data.blobName.blob.core.windows.net':blobKey})
except:
  dbutils.fs.unmount(mount_point='/mnt/data1')
  dbutils.fs.mount(source=blobEndpoint,mount_point='/mnt/data1',extra_configs={'fs.azure.sas.data.blobName.blob.core.windows.net':blobKey})

In [0]:
# Configuración necesaria para poder guardar archivos en el storage account
spark.conf.set ( "fs.azure.account.key.blobName.dfs.core.windows.net",
  "blobKey")

## Lectura y unión de distintas fuentes de información

In [0]:
# Se leen los archivos de peticiones guardados en el contenedor del Storage Account
DF_2015 = spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2015A.txt")
# Se guarda en una lista las columnas que tiene df de 2015, de forma que todos los demás dataframes tengan el mismo orden de columnas (requerido para hacer union)
vars_2015 = DF_2015.columns
DF_2016= spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2016A.txt").select(vars_2015)
DF_2017= spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2017A.txt").select(vars_2015)
DF_2018= spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2018A.txt").select(vars_2015)
DF_2019= spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2019A.txt").select(vars_2015)
DF_2020= spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2020A.txt").select(vars_2015)
DF_2021= spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/0101-30062021A.txt").select(vars_2015)

# Finalmente se hace la unión de todos los años en un solo DF
DF_DENUNCIAS = DF_2015.union(DF_2016).union(DF_2017).union(DF_2018).union(DF_2019).union(DF_2020).union(DF_2021)
DF_DENUNCIAS.head()

In [0]:
# Lectura de datos PARD
DF_PARD = spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'utf8').csv("/mnt/data1/Data/13072021/CONSOL UNIFICADO INGRESOS PARD AGOSTO2011_2019_MAYO 2021.csv")

In [0]:
#Cargamos las tablas con casos de RAVD y las unimos.
# Hay datos desde 01/01/2015 hasta 31/05/2021
DF_2015 = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].['2015_2$']").option("user", jdbcuser).option("password", jdbcpass).load()
# Se guarda en una lista las columnas que tiene df de 2015, de forma que todos los demás dataframes tengan el mismo orden de columnas (requerido para hacer union)
vars_2015 = DF_2015.columns
DF_2016 = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].['2016_2$']").option("user", jdbcuser).option("password", jdbcpass).load().select(vars_2015)
DF_2017 = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].['2017_2$']").option("user", jdbcuser).option("password", jdbcpass).load().select(vars_2015)
DF_2018 = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].['2018_2$']").option("user", jdbcuser).option("password", jdbcpass).load().select(vars_2015)
DF_2019 = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].['2019_2$']").option("user", jdbcuser).option("password", jdbcpass).load().select(vars_2015)
DF_2020 = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].['2020_2$']").option("user", jdbcuser).option("password", jdbcpass).load().select(vars_2015)
DF_2021 = spark.read.format("com.microsoft.sqlserver.jdbc.spark").option("url", jdbcUrl).option("dbtable", "[dbo].['2021_2$']").option("user", jdbcuser).option("password", jdbcpass).load().select(vars_2015)
DF_RAVD=DF_2015.union(DF_2016).union(DF_2017).union(DF_2018).union(DF_2019).union(DF_2020).union(DF_2021)
DF_RAVD.columns

In [0]:
# Datos de Inobservancia de Derechos desde 01/01/2018
DF_Inobservancia = spark.read.option("header", "true").option('delimiter','\t').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/PetsInobDer01012018-30062021.txt")

In [0]:
# Se leen los archivos que contienen los campos de PaisAfectado y PaisPeticionario
cols_pais = ["NúmeroPetición", "PaisResidenciaPeticionario", "PaisResidenciaAfectado"]
DF_2015 = spark.read.option("header", "true").option('delimiter','|').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2015_TipPetDem.txt").select(cols_pais)
DF_2016= spark.read.option("header", "true").option('delimiter','|').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2016_TipPetDem.txt").select(cols_pais)
DF_2017= spark.read.option("header", "true").option('delimiter','|').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2017_TipPetDem.txt").select(cols_pais)
DF_2018= spark.read.option("header", "true").option('delimiter','|').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2018_TipPetDem.txt").select(cols_pais)
DF_2019= spark.read.option("header", "true").option('delimiter','|').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2019_DepPetAfec.txt").select(cols_pais)
DF_2020= spark.read.option("header", "true").option('delimiter','|').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2020_DepPetAfec.txt").select(cols_pais)
DF_2021= spark.read.option("header", "true").option('delimiter','|').option('encoding', 'latin1').csv("/mnt/data1/Data/13072021/2021_DepPetAfec.txt").select(cols_pais)


# Finalmente se hace la unión de todos los años en un solo DF
DF_PAIS = DF_2015.union(DF_2016).union(DF_2017).union(DF_2018).union(DF_2019).union(DF_2020).union(DF_2021)

In [0]:
# Archivo de valoraciones PARD
DF_VALORACIONES = spark.read.option("header", "true").option('delimiter',';').option('encoding', 'latin1').csv("/mnt/data1/Data/SRD-AVD_01012019_02082021_AJUSTADO.csv")

## Join de la base de denuncias con las otras fuentes de información

#### 1. Union de DF_Inobservancia a DF_DENUNCIAS

In [0]:
# El archivo de DF_Inobservancia también trae una columna llamada "fechaNacimientoPeticionario" que no tienen todos los archivos de DF_DENUNCIAS, por lo que es necesario remover esta columna
DF_Inobservancia = DF_Inobservancia.drop('fechaNacimientoPeticionario')

# Ahora si se puede hacer la unión sin errores
DF_UNIDO_1 = DF_DENUNCIAS.union(DF_Inobservancia)

#### 2. Eliminación de registros que tienen valores inválidos en la llave con la que se hacen los cruces "NúmeroPetición"

In [0]:
# Verificamos la cantidad de ID nulos
print("ID NULOS: ", DF_UNIDO_1.filter(col("NúmeroPetición").isNull()).count())

#Verificamos cuántos registros tienen ID duplicados y cuál es el número de ID duplicados
ID_duplicados = DF_UNIDO_1.groupby("NúmeroPetición").count().filter(col("count")>1).cache()
print("Número de registros con ID duplicados: ", ID_duplicados.agg({'count': 'sum'}).collect()[0][0], 
      "Numero de ID que están duplicados: ", ID_duplicados.count())

# Mostramos los primeros 10 ID más repetidos
print(ID_duplicados.sort("count", ascending=False).show(20))

ID NULOS: 0
Número de registros con ID duplicados: 2281 Numero de ID que están duplicados: 9
+--------------+-----+
NúmeroPetición|count|
+--------------+-----+
 1,76127E+11| 1940|
 1,76155E+11| 327|
 1762251547| 2|
 30477310| 2|
 17923145| 2|
 27121781| 2|
 18207419| 2|
 17923159| 2|
 27121782| 2|
+--------------+-----+

None

Se encuentra que en particular hay dos ID que están repetidos muchas veces: 1,76127E+11 y 1,76155E+11. Se profundiza en saber si es un error de lectura de los datos originales

In [0]:
# Imprimir número total de registros que tienen alguno de estos dos ID
ID_raros = DF_UNIDO_1.filter((col("NúmeroPetición")=="1,76127E+11") | (col("NúmeroPetición")=="1,76155E+11")).groupby("AñoRegistroPetición", "NúmeroPetición").count()
print("Número de registros con ID raros: ", ID_raros.agg({'count': 'sum'}).collect()[0][0])
ID_raros.sort("count", ascending=False).show()

Número de registros con ID raros: 2267
+-------------------+--------------+-----+
AñoRegistroPetición|NúmeroPetición|count|
+-------------------+--------------+-----+
 2020| 1,76127E+11| 1148|
 2021| 1,76127E+11| 549|
 2021| 1,76155E+11| 325|
 2019| 1,76127E+11| 243|
 2020| 1,76155E+11| 2|
+-------------------+--------------+-----+

Viendo los archivos fuente se encuentra que hay un error en estos, desde la fuente de información entregada por SyA. Estos dos ID raros se repiten y se encuentran tal cual en los archivos txt recibidos. Debido a que estos registros no se pueden rastrear y tampoco cruzar exactamente con PARD, deben ser eliminados.

In [0]:
print("Número de registros antes de filtro de ID raros: ", DF_UNIDO_1.count())
DF_UNIDO_1 = DF_UNIDO_1.filter((col("NúmeroPetición")!="1,76127E+11") & (col("NúmeroPetición")!="1,76155E+11"))
print("Número de registros después de filtro de ID raros: ", DF_UNIDO_1.count())

Número de registros antes de filtro de ID raros: 1322180
Número de registros después de filtro de ID raros: 1319913

Ahora se revisarán los otros duplicados. Si tienen el mismo ID pero corresponden a diferentes peticiones o afectados, son errores de la base de datos y deben ser también eliminados. También pueden ser duplicados exactos del mismo registro, por lo cual debe permanecer solo uno de los registros. Luego de una verificación manual, se encuentra que son duplicados exactos, por lo que debe permanecer solo uno. Para ello usamos la función dropDuplicates en la columna "NúmeroPetición"

In [0]:
print("Número de registros antes de filtro de ID duplicados: ", DF_UNIDO_1.count())
DF_UNIDO_1 = DF_UNIDO_1.dropDuplicates(subset=["NúmeroPetición"])
print("Número de registros después de filtro de ID duplicados: ", DF_UNIDO_1.count())

Número de registros antes de filtro de ID duplicados: 1319913
Número de registros después de filtro de ID duplicados: 1319906

In [0]:
# Verificamos finalmente que no haya duplicados ni nulos en el ID NumeroPeticion
# Verificamos la cantidad de ID nulos
print("ID NULOS: ", DF_UNIDO_1.filter(col("NúmeroPetición").isNull()).count())

#Verificamos cuántos registros tienen ID duplicados y cuál es el número de ID duplicados
ID_duplicados = DF_UNIDO_1.groupby("NúmeroPetición").count().filter(col("count")>1).cache()
print("Número de registros con ID duplicados: ", ID_duplicados.agg({'count': 'sum'}).collect()[0][0], 
      "Numero de ID que están duplicados: ", ID_duplicados.count())

# Mostramos los primeros 10 ID más repetidos
print(ID_duplicados.sort("count", ascending=False).show(10))

ID NULOS: 0
Número de registros con ID duplicados: None Numero de ID que están duplicados: 0
+--------------+-----+
NúmeroPetición|count|
+--------------+-----+
+--------------+-----+

None

#### 3. Join DF_DENUNCIAS con DF_RAVD

In [0]:
#Seleccionamos los atributos que se requieren y hacemos el cruce con el DF que hemos cruzado previamente
DF_RAVD = DF_RAVD.select('Petición','D_CONSTATADAS','CONSTATACION','Val_Constatacion','RAVD_Constatadas')
DF_RAVD = DF_RAVD.withColumnRenamed('Petición','NúmeroPetición') # Renombramos nombre de columna para poder hacer cruce
DF_UNIDO_2 = DF_UNIDO_1.join(DF_RAVD,on='NúmeroPetición',how="left")

In [0]:
# Se crea una función para verificar el número de registros cruzados y el estatus de cruce (cruzó, no cruzó-registro solo en base izquierda, no cruzó-registro solo en base derecha)
def count_join_results(left_df, right_df, col_join):
  left_df_2 = left_df.select(col_join)
  left_df_2 = left_df_2.withColumn("left", F.lit("left_only"))
  right_df_2 = right_df.select(col_join)
  right_df_2 = right_df_2.withColumn("right", F.lit("right_only"))

  count_cruce = left_df_2.join(right_df_2,on=col_join,how="fullouter")
  count_cruce = count_cruce.withColumn("merge_result", 
                                      when((col("left")=="left_only") & (col("right").isNull()), 
                                          "left_only") \
                                      .when((col("left").isNull()) & (col("right")=="right_only"), 
                                           "right_only") \
                                      .otherwise("merged"))

  print("Left total rows: ", left_df_2.count(), ". Right total rows: ", right_df_2.count())
  print(count_cruce.groupby("merge_result").count().show())

In [0]:
count_join_results(DF_UNIDO_1, DF_RAVD, "NúmeroPetición")

Left total rows: 1319906 . Right total rows: 391131
+------------+------+
merge_result| count|
+------------+------+
 right_only| 50|
 left_only|928825|
 merged|391081|
+------------+------+

None

De las RAVD solo dejaron de cruzar 50 registros, lo que es un porcentaje bastante pequeño (0,013%). Ahora debemos verificar todas los registros TipoPetición == RAVD del DF_DENUNCIAS hayan cruzado con el DF_RAVD, solo las SRD o Inobservancia no debieron cruzar

In [0]:
# Verificamos que las que no cruzaron en DF_DENUNCIAS son solo SRD. Esto lo hacemos con la columna "CONSTATACION" que solo está en DF_RAVD
DF_UNIDO_2.select("TipoPetición", "CONSTATACION").groupby("TipoPetición", "CONSTATACION").count().sort("TipoPetición").show(truncate=False)

+-----------------------------------------------+-----------------------------+------+
TipoPetición |CONSTATACION |count |
+-----------------------------------------------+-----------------------------+------+
Inobservancia de derechos |null |8614 |
Reporte de Amenaza o Vulneración de derechos |Fallida |2118 |
Reporte de Amenaza o Vulneración de derechos |Dirección Errada |91525 |
Reporte de Amenaza o Vulneración de derechos |Remitido a Otra Entidad |67394 |
Reporte de Amenaza o Vulneración de derechos |Sin Resultado de Constatación|6353 |
Reporte de Amenaza o Vulneración de derechos |null |4815 |
Reporte de Amenaza o Vulneración de derechos |Verdadera |79989 |
Reporte de Amenaza o Vulneración de derechos |Falsa |143700|
Solicitud de Restablecimiento de Derechos (SRD)|null |915396|
Solicitud de Restablecimiento de Derechos (SRD)|Sin Resultado de Constatación|1 |
Solicitud de Restablecimiento de Derechos (SRD)|Verdadera |1 |
+-----------------------------------------------+-----------------------------+------+

In [0]:
# Hay 4.815 RAVD que no cruzaron. Puede ser por los diferentes cortes que tienen las bases
DF_UNIDO_2.filter((col("TipoPetición")=="Reporte de Amenaza o Vulneración de derechos") & (col("CONSTATACION").isNull())) \
.select("AñoRegistroPetición", "MesRegistroPetición").groupby("AñoRegistroPetición", "MesRegistroPetición").count().sort("count", ascending=False).show(truncate=False)

+-------------------+-------------------+-----+
AñoRegistroPetición|MesRegistroPetición|count|
+-------------------+-------------------+-----+
2021 |Jun |4758 |
2019 |Apr |4 |
2015 |Jul |4 |
2018 |Jan |4 |
2015 |Mar |3 |
2016 |Oct |3 |
2016 |Jun |3 |
2018 |May |3 |
2016 |Mar |3 |
2015 |Jun |3 |
2015 |Feb |2 |
2016 |Feb |2 |
2015 |Jan |2 |
2016 |May |2 |
2019 |Jun |2 |
2015 |Sep |1 |
2020 |Nov |1 |
2019 |Jul |1 |
2020 |Jul |1 |
2018 |Apr |1 |
+-------------------+-------------------+-----+
only showing top 20 rows

In [0]:
# Efectivamente, 4.758 no cruzan porque corresponden a junio-2021, mientras que la información obtenida de RAVD solo está hasta mayo-2021.
# A continuación dejamos solo información hasta mayo-2021
DF_UNIDO_2 = DF_UNIDO_2.filter((col("AñoRegistroPetición") != "2021") | (col("MesRegistroPetición") != "Jun"))

#### 4. Join DF_DENUNCIAS con DF_PARD

In [0]:
#Seleccionamos los atributos que se requieren y hacemos el cruce con el DF que hemos cruzado previamente
DF_PARD = DF_PARD.select("CODIGO CASO", "TIPO_PETICION", "PRD_160 FECHA_APERTURA", "ANIO","MES", "UNFICADO MOTIVO INGRESO", "PRD_760_CIERRE_PROCESO_RESTABLECIMIENTO", "PA/PC", "FECHA ACTUACION MAS RECIENTE", "MEDIDA ACTUACION MAS RECIENTE", "MEDIDA_TOMADA_ULTIMA", "PRD_165")
DF_PARD = DF_PARD.withColumnRenamed('CODIGO CASO','NúmeroPetición') # Renombramos nombre de columna para poder hacer cruce
DF_UNIDO_3 = DF_UNIDO_2.join(DF_PARD,on='NúmeroPetición',how="left")

In [0]:
count_join_results(DF_UNIDO_2, DF_PARD, "NúmeroPetición")

Left total rows: 1302971 . Right total rows: 407948
+------------+-------+
merge_result| count|
+------------+-------+
 right_only| 139332|
 left_only|1034540|
 merged| 268616|
+------------+-------+

None

Debemos verificar que los que no unieron a derecha son PARD que entran por un TipoPetición diferente a SRD, RAVD o Inobservancia

In [0]:
# Creamos un full outer join que nos recupere todas las columnas y además cuáles cruzaron, cuáles registros solo están en DF_UNIDO y cuáles registros están solo en DF_PARD
count_cruce = DF_UNIDO_2.withColumn("left", F.lit("left_only")).join(DF_PARD.withColumn("right", F.lit("right_only")),on="NúmeroPetición",how="fullouter")
count_cruce = count_cruce.withColumn("merge_result", 
                                    when((col("left")=="left_only") & (col("right").isNull()), 
                                        "left_only") \
                                    .when((col("left").isNull()) & (col("right")=="right_only"), 
                                         "right_only") \
                                    .otherwise("merged"))

In [0]:
# Hacemos un groupby del resultado del cruce y el tipo de petición
count_cruce.groupby("merge_result", "TIPO_PETICION").count().sort("merge_result", "TIPO_PETICION").show(200, truncate=False)

+------------+---------------------------------------------------------------------+-------+
merge_result|TIPO_PETICION |count |
+------------+---------------------------------------------------------------------+-------+
left_only |null |1034540|
merged |null |214 |
merged |Denuncias PRD |17502 |
merged |FISCALIA |1 |
merged |Inobservancia de derechos |89 |
merged |NairAstridCamejoOrozco |1 |
merged |Reporte de Amenaza o Vulneración de derechos |8541 |
merged |Solicitud de Restablecimiento de Derechos (SRD) |242268 |
right_only |null |964 |
right_only |Asesoría en Derecho de Familia |7 |
right_only |Asistencia y Asesoría a la Familia |367 |
right_only |Asistencia y Asesoría a la niñez y la Familia (AANF) |7350 |
right_only |Atención en Crisis |2 |
right_only |Denuncias PRD |85224 |
right_only |Denuncias Violencia Intrafamiliar |67 |
right_only |Derecho de Petición - Información y Orientación |654 |
right_only |Derecho de Petición - Información y Orientación con Trámite |19 |
right_only |Derecho de Petición - Quejas |1 |
right_only |Derecho de Petición - Reclamos |1 |
right_only |Derechos de Petición - Denuncias PRD |31 |
right_only |Informacion y Orientacion con Tramite |240 |
right_only |NNA Desvinculados Grupos Armados |4 |
right_only |PRD - Asuntos No Conciliables |1589 |
right_only |PRD- Asuntos Conciliables |162 |
right_only |Peticiones Recibidas en Otros Niveles |17 |
right_only |Proceso conflicto con la ley (PCL) |1349 |
right_only |Reporte de Amenaza o Vulneración de derechos |38 |
right_only |Solicitud de Cupo |465 |
right_only |Solicitud de Restablecimiento de Derechos (SRD) |39237 |
right_only |Solicitud de Restablecimiento de Derechos– Otras Autoridades (SRD_OA)|237 |
right_only |Transferencias |1 |
right_only |Trámite de Adopción |18 |
right_only |Trámite de atención Extraprocesal (TAE) |1288 |
+------------+---------------------------------------------------------------------+-------+

Se observa que hay 38 RAVD que están en DF_PARD pero no cruzan con DF_DENUNCIAS, no es mucho entonces no es algo para profundizar. Sin embargo, hay 39.237 SRD que están en DF_PARD pero no en DF_DENUNCIAS, lo que corresponde al 13,94% de las 281.509 SRD en DR_PARD (39.237+242.272). Debido a que es un porcentaje alto debemos investigar de dónde provienen estos que no cruzan

In [0]:
PARD_SRD_nocruzan = count_cruce.filter((col("merge_result") == "right_only") & 
                                      (col("TIPO_PETICION") == "Solicitud de Restablecimiento de Derechos (SRD)"))
PARD_SRD_nocruzan.groupby("ANIO").count().sort("count", ascending=False).show(200)

+----+-----+
ANIO|count|
+----+-----+
2014|25745|
2013|10461|
2015| 1538|
2020| 452|
2021| 306|
2016| 285|
2012| 104|
2018| 99|
2017| 99|
2011| 86|
2019| 62|
+----+-----+

Se encuentra que la gran mayoría de registros que no cruzan son del periodo 2011-2014, lo cual tiene sentido porque la base de denuncas DF_DENUNCIAS solo contiene información desde 2015. A partir de esto, los PARD abiertos por una petición SRD que no cruzan son 2.841 lo que representa un 1,01% del total.

#### 5. Cruce para recuperar campo de PaisAfectado y PaisPeticionario

In [0]:
DF_UNIDO_4 = DF_UNIDO_3.join(DF_PAIS,on='NúmeroPetición',how="left")
count_join_results(DF_UNIDO_3, DF_PAIS, "NúmeroPetición")

Left total rows: 1303156 . Right total rows: 1332448
+------------+-------+
merge_result| count|
+------------+-------+
 right_only| 37045|
 left_only| 7573|
 merged|1295588|
+------------+-------+

None

Verificamos las razones por las cuales hay unos registros que no cruzan, ya sea porque solo están en la base de peticiones (izquierda, 7.573 registros) o solo están en la base de PAIS (derecha, 37.045)

In [0]:
# Creamos un full outer join que nos recupere todas las columnas y además cuáles cruzaron, cuáles registros solo están en DF_UNIDO_4 y cuáles registros están solo en DF_PAIS
count_cruce = DF_UNIDO_3.withColumn("left", F.lit("left_only")).join(DF_PAIS.withColumn("right", F.lit("right_only")),on="NúmeroPetición",how="fullouter")

count_cruce = count_cruce.withColumn("merge_result", 
                                    when((col("left")=="left_only") & (col("right").isNull()), 
                                        "left_only") \
                                    .when((col("left").isNull()) & (col("right")=="right_only"), 
                                         "right_only") \
                                    .otherwise("merged"))

count_cruce.groupby("merge_result", "AñoRegistroPetición").count().sort("merge_result", "AñoRegistroPetición").show(200, truncate=False)

+------------+-------------------+------+
merge_result|AñoRegistroPetición|count |
+------------+-------------------+------+
left_only |2016 |12 |
left_only |2018 |85 |
left_only |2019 |4963 |
left_only |2020 |2507 |
left_only |2021 |6 |
merged |2015 |146730|
merged |2016 |166601|
merged |2017 |214404|
merged |2018 |253260|
merged |2019 |226614|
merged |2020 |202289|
merged |2021 |85690 |
right_only |null |37045 |
+------------+-------------------+------+

Se encuentra que la gran mayoría de registros que solo están a izquierda corresponden a 2019 (4.963) y 2020 (2.507). Debido a que no se conoce la razón de que no se haya encontrado la información de PaisAfectado y PaisPeticionario, para estos registros se dejará la información de país como "NA".

De los otros 37.045 registros que no cruzan a derecha no se puede saber su origen, debido a que los archivos fuente solo traen las columnas de NúmeroPetición, PaisResidenciaPeticionario y PaisResidencia Afectado. Sin embargo, lo más probable es que sean registros muy recientes, posteriores a mayo del 2021, por lo que no cruzarán con las bases de peticiones que solo tienen información hasta mayo-2021.

#### 6. Cruce para recuperar variables de valoraciones PARD

In [0]:
#Seleccionamos los atributos que se requieren y hacemos el cruce con el DF que hemos cruzado previamente
DF_VALORACIONES = DF_VALORACIONES.select('codigoCaso', 'fechaRegistroPeticion', 'fecha_ingreso', 'FechaAEG_013', 'FechaPRD_500', 'FechaPRD_510', 'FechaPRD_525', 'FechaPRD_045', 'FechaPRD_825', 'PrimeraFechaPRD_160', 'FechaPRD_165', 'FechaPRD_845', 'PRD_845_AD')

DF_VALORACIONES = DF_VALORACIONES.withColumnRenamed('codigoCaso','NúmeroPetición') # Renombramos nombre de columna para poder hacer cruce
DF_VALORACIONES = DF_VALORACIONES.withColumnRenamed('fechaRegistroPeticion','fechaRegistroPeticion_val')

DF_UNIDO_5 = DF_UNIDO_4.join(DF_VALORACIONES,on='NúmeroPetición',how="left")
count_join_results(DF_UNIDO_4, DF_VALORACIONES, "NúmeroPetición")

Left total rows: 1303161 . Right total rows: 429695
+------------+------+
merge_result| count|
+------------+------+
 right_only| 37324|
 left_only|910761|
 merged|392400|
+------------+------+

None

In [0]:
# Crear columna con año registro petición del DF_VALORACIONES
DF_VALORACIONES = DF_VALORACIONES.withColumn("agno_regpeticion", F.substring('fechaRegistroPeticion_val', 7,4))
DF_VALORACIONES = DF_VALORACIONES.withColumn("mes_regpeticion", F.substring('fechaRegistroPeticion_val', 4,2))

# Se elimina variable de tabla DF_UNIDO_5 que ya no es necesaria
DF_UNIDO_5 = DF_UNIDO_5.drop("fechaRegistroPeticion_val")

# Creamos un full outer join que nos recupere todas las columnas y además cuáles cruzaron, cuáles registros solo están en DF_UNIDO_5 y cuáles registros están solo en DF_VALORACIONES
count_cruce = DF_UNIDO_4.withColumn("left", F.lit("left_only")).join(DF_VALORACIONES.withColumn("right", F.lit("right_only")),on="NúmeroPetición",how="fullouter")

count_cruce = count_cruce.withColumn("merge_result", 
                                    when((col("left")=="left_only") & (col("right").isNull()), 
                                        "left_only") \
                                    .when((col("left").isNull()) & (col("right")=="right_only"), 
                                         "right_only") \
                                    .otherwise("merged"))

count_cruce.groupby("merge_result", "AñoRegistroPetición", "agno_regpeticion").count().sort("merge_result", "AñoRegistroPetición", "agno_regpeticion").show(200, truncate=False)

+------------+-------------------+----------------+------+
merge_result|AñoRegistroPetición|agno_regpeticion|count |
+------------+-------------------+----------------+------+
left_only |2015 |null |146730|
left_only |2016 |null |166613|
left_only |2017 |null |214404|
left_only |2018 |null |253345|
left_only |2019 |null |55695 |
left_only |2020 |null |54436 |
left_only |2021 |null |19538 |
merged |2019 |2019 |175882|
merged |2020 |2020 |150358|
merged |2020 |2021 |2 |
merged |2021 |2021 |66158 |
right_only |null |2019 |3694 |
right_only |null |2020 |4081 |
right_only |null |2021 |29549 |
+------------+-------------------+----------------+------+

Se observa que el archivo de valoraciones solo tiene registros entre 2019 y 2021. Todos los registros de valoraciones (429.695) deberían haber cruzado con la base de peticiones - DF_UNIDO_4. Sin embargo, hay 37.324 registros que no cruzan, que en su mayoría corresponden a 2021 (29.549, o 79% de los 37.324). Puede que estos sean registros de meses posteriores a mayo-2021, meses que fueron eliminados de la base de denuncias en una celda anterior. A continuación verificamos los meses de la base de valoraciones que no cruzan (right_only)

In [0]:
count_cruce.filter((col("agno_regpeticion")=="2021") & (col("merge_result")=="right_only")).groupby("merge_result", "agno_regpeticion","mes_regpeticion").count().sort(["merge_result", "agno_regpeticion", "mes_regpeticion"]).show()

+------------+----------------+---------------+-----+
merge_result|agno_regpeticion|mes_regpeticion|count|
+------------+----------------+---------------+-----+
 right_only| 2021| 01| 300|
 right_only| 2021| 02| 328|
 right_only| 2021| 03| 403|
 right_only| 2021| 04| 359|
 right_only| 2021| 05| 280|
 right_only| 2021| 06|13775|
 right_only| 2021| 07|14104|
+------------+----------------+---------------+-----+

Efectivamente la mayoría de los registros que no cruzan corresponden a meses posteriores a mayo-2021. De esta forma, solo es un pequeño porcentaje (2,12%) de los registros de valoraciones que no cruzan con la base de peticiones.

#### 7. Limpieza de nombres de variables
Es necesario limpiar los nombres de las variables (quitar espacios, carácteres especiales, entre otros) para poder exportar sin problemas los datos

In [0]:
names_columns = DF_UNIDO_5.columns

names_columns = list(map(lambda string: \
                    normalize("NFKD", string).encode("ascii","ignore").decode("ascii") \
                        .replace('\n', ' ').replace(' ', '_').replace('(', '').replace(')', '') \
                    , names_columns))

DF_UNIDO_5 = DF_UNIDO_5.toDF(*names_columns)

#### 8. Se escribe en el Storage Account la base final cruzada

In [0]:
# Guardar como csv
file_location = "abfss://basecruzada@blobName.dfs.core.windows.net/basecruzada.csv"
DF_UNIDO_5.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").mode("overwrite").save(file_location)

In [0]:
# Guardar como parquet
file_location = "abfss://basecruzada@blobName.dfs.core.windows.net/basecruzada.parquet"
DF_UNIDO_5.coalesce(1).write.option("header", "true").mode("overwrite").parquet(file_location)